In [ ]:
import pandas as pd
from Bio import Entrez
import os
import time

In [ ]:
df = pd.read_csv("ecoli_only_100000.csv")
# Konfiguracja Entrez
Entrez.email = email
# Upewnij się, że Twój klucz API jest poprawny
Entrez.api_key = apikey

# Lista Twoich numerów akcesyjnych
accessions = list(df.accession.unique())

In [ ]:
# Folder na wyniki
output_dir = "ecoli_genomes"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


def download_sequences(acc_list):
    print(f"Rozpoczynam pobieranie {len(acc_list)} sekwencji...")

    for acc in acc_list:
        file_path = os.path.join(output_dir, f"data/complete_ecoli/{acc}.fasta")

        # Pomijamy, jeśli plik już istnieje
        if os.path.exists(file_path):
            print(f"[*] Plik {acc}.fasta już istnieje, pomijam.")
            continue

        try:
            print(f"[>] Pobieranie {acc}...")
            # efetch pobiera dane bezpośrednio z bazy
            handle = Entrez.efetch(
                db="nucleotide", id=acc, rettype="fasta", retmode="text"
            )
            fasta_data = handle.read()
            handle.close()

            # Zapis do pliku
            with open(file_path, "w") as f:
                f.write(fasta_data)

            print(f"[OK] Zapisano w {file_path}")

            # Przerwa dla serwerów NCBI (max 3 zapytania na sekundę bez API Key)
            time.sleep(0.5)

        except Exception as e:
            print(f"[BŁĄD] Nie udało się pobrać {acc}: {e}")


download_sequences(accessions)
print("\nProces zakończony.")